# 🛡️S2W5总结：Transformer核心闯关

**目标**：不依赖文档，写出Transformer的核心组件，并修复一个故意留下的Bug。

**准备工作**：

In [6]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# 检查环境
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前复习环境：{device}")

当前复习环境：cuda


## 1 手写Attention

**挑战描述**：`scaled_dot_product_attention`是所有大模型的基石，我们需要补充完整函数体中的核心步骤。

**考察点**：
1. 转置`transpose`的维度。
2. 缩放`sqrt`的位置。
3. Softmax的维度。

In [7]:
def scaled_dot_product_attention(query, key, value, mask=None):
    """
    Inputs:
        query, key, value: [Batch, Heads, Seq_Len, D_k]
    """
    d_k = query.size(-1)
    
    # --- 填空 1: 计算分数 (Score) ---
    # 提示: Q 乘以 K的转置。注意 K 需要转置最后两个维度。
    scores = torch.matmul(query, key.transpose(-2, -1))

    # --- 🕳️ 填空 2: 缩放 (Scaling) ---
    # 提示: 除以 d_k 的平方根
    scores = scores / math.sqrt(d_k)

    # --- (可选) Mask 步骤 ---
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    # --- 🕳️ 填空 3: 归一化 (Softmax) ---
    # 提示: 在最后一个维度上进行
    attn_weights = F.softmax(scores, dim=1)

    # --- 🕳️ 填空 4: 加权求和 (Weighted Sum) ---
    # 提示: 权重 乘以 Value
    output = torch.matmul(attn_weights, value)

    return output, attn_weights

# -------- 验证代码 ---------------
q = torch.randn(2, 4, 10, 32) # [B, H, L, D]
k = torch.randn(2, 4, 10, 32)
v = torch.randn(2, 4, 10, 32)

out, w = scaled_dot_product_attention(q, k, v)
print("Level 1 Output Shape:", out.shape)
assert out.shape == (2, 4, 10, 32), "维度不对，请检查填空！"
print("🎉 Level 1 通关！")

Level 1 Output Shape: torch.Size([2, 4, 10, 32])
🎉 Level 1 通关！


## 2 Multi-Head切分

**挑战描述**：多头注意力的核心在于维度的`view`和`permute`。这是最容易晕的地方。

**考察点**：如何把`[Batch, Seq, D_model]`变成`[Batch, Heads, Seq, D_k]`。

In [8]:
class HeadSplitter(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.n_heads = n_heads
        self.d_k = d_model // n_heads

    def forward(self, x):
        batch_size = x.size(0)

        # --- 🕳️ 填空 5: 维度重塑 (View) ---
        # 提示: 把 d_model 拆成 n_heads 和 d_k
        x = x.view(batch_size, -1, self.n_heads, self.d_k)

        # --- 🕳️ 填空 6: 维度交换 (Permute) ---
        # 提示: 我们希望 Heads 维度排在 Seq 维度前面，方便并行计算
        # 目标形状: [Batch, Heads, Seq, D_k]
        x = x.permute(0, 2, 1, 3)

        return x
    
# --- ✅ 验证代码 ---
splitter = HeadSplitter(d_model=512, n_heads=8)
dummy_x = torch.randn(32, 10, 512) # [Batch, Seq, D_model]
split_x = splitter(dummy_x)
print("Level 2 Output Shape:", split_x.shape)
assert split_x.shape == (32, 8, 10, 64), "维度切分错误！"
print("🎉 Level 2 通关！")

Level 2 Output Shape: torch.Size([32, 8, 10, 64])
🎉 Level 2 通关！


## 3 Debug Challenge

**挑战描述**：下面这段 EncoderLayer 的代码有一个致命 Bug，虽然能运行，但丢失了残差连接的意义。请找出它，并修复。

**场景**：老王写了个 Encoder，但他发现模型越训练越差，梯度好像传不回去。

In [ ]:
class BrokenEncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, n_heads) # 这里偷懒用了 PyTorch 自带的
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model)
        )
        
    def forward(self, x):
        # x shape: [Seq, Batch, Dim] (PyTorch官方实现默认Seq在前)
        
        # 1. Attention
        attn_output, _ = self.self_attn(x, x, x)
        
        # --- 🐞 Bug 在这里！请观察数据流 ---
        # 原始代码:
        # x = self.norm1(attn_output) 
        
        # ❌ 错误原因: 这里的 x 直接被 attn_output 覆盖了！
        # 原始的输入 x (残差) 被丢弃了。这会导致梯度消失。
        
        # ✅ 请修正为: (Add & Norm)
        x = self.norm1(x + attn_output)
        # -------------------------------
        
        # 2. FFN
        ffn_output = self.ffn(x)
        
        # --- 🐞 同样的逻辑，下面这里也要修 ---
        # 原始代码:
        x = self.norm2(ffn_output)
        
        # ✅ 请修正为:
        # x = self.norm2(x + ffn_output)
        # -------------------------------
        
        return x

# --- ✅ 验证代码 ---
layer = BrokenEncoderLayer(d_model=512, n_heads=8)
x = torch.randn(10, 32, 512)
out = layer(x)
print("Level 3 Output Shape:", out.shape)
print("🎉 Level 3 (找茬) 通关！残差连接已修复。")

Level 3 Output Shape: torch.Size([10, 32, 512])
🎉 Level 3 (找茬) 通关！残差连接已修复。
